# Fase 4: Monitoreo y Detección de Data Drift
# Marketing Campaign Response Prediction

---

## Objetivo

Este notebook implementa el monitoreo del modelo y detección de data drift:

1. **Cálculo de métricas de drift**: PSI, KS Test, Jensen-Shannon, Chi-cuadrado
2. **Detección de cambios**: Comparación baseline vs datos actuales
3. **Visualización**: Gráficos de distribución y métricas
4. **Alertas**: Sistema de alertas por umbrales
5. **Recomendaciones**: Sugerencias de retraining

---

## 1. Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import ks_2samp, chi2_contingency
from scipy.spatial.distance import jensenshannon
import warnings
warnings.filterwarnings('ignore')

# Importar funciones de model_monitoring.py
import sys
sys.path.append('.')
from model_monitoring import (
    calculate_psi,
    calculate_ks_test,
    calculate_js_divergence,
    calculate_chi_square,
    detect_drift,
    get_drift_status
)

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print('✅ Librerías importadas correctamente')

## 2. Carga de Datos

In [ ]:
# Cargar datos baseline (históricos)
# Estos son los datos con los que se entrenó el modelo
try:
    baseline_data = pd.read_csv('../../data_processed.csv')
    print('✅ Datos baseline cargados')
    print(f'   Shape: {baseline_data.shape}')
    print(f'   Columnas: {baseline_data.columns.tolist()}')
except FileNotFoundError:
    print('⚠️ No se encontró data_processed.csv')
    print('   Intentando cargar desde X_train_transformed.csv...')
    try:
        baseline_data = pd.read_csv('../../X_train_transformed.csv')
        print('✅ Datos baseline cargados desde X_train_transformed.csv')
    except FileNotFoundError:
        print('❌ Error: No se encontraron datos baseline')
        print('   Ejecuta primero la Fase 2 para generar los datos procesados')

In [ ]:
# Cargar datos actuales
# En producción, estos serían datos nuevos que llegan continuamente
# Para este ejemplo, usaremos una muestra de los datos baseline (simulación)

# Simulación: Tomar una muestra de los datos baseline
# En producción, aquí cargarías datos reales nuevos
if 'baseline_data' in locals():
    # Crear datos actuales simulados (50% de los datos baseline)
    current_data = baseline_data.sample(frac=0.5, random_state=42)
    
    # Simular algún cambio (opcional)
    # Por ejemplo, agregar ruido a algunas variables numéricas
    numeric_cols = baseline_data.select_dtypes(include=[np.number]).columns
    for col in numeric_cols[:3]:  # Solo las primeras 3 variables numéricas
        noise = np.random.normal(0, baseline_data[col].std() * 0.1, len(current_data))
        current_data[col] = current_data[col] + noise
    
    print('✅ Datos actuales simulados')
    print(f'   Shape: {current_data.shape}')
else:
    print('❌ Error: Primero carga los datos baseline')

## 3. Cálculo de Métricas de Drift

### Métricas implementadas:

1. **PSI (Population Stability Index)**: Mide el cambio en la distribución
   - PSI < 0.1: Sin cambio significativo
   - 0.1 ≤ PSI < 0.2: Cambio moderado
   - PSI ≥ 0.2: Cambio significativo

2. **KS Test (Kolmogorov-Smirnov)**: Compara distribuciones
   - Estadístico: Distancia máxima entre distribuciones
   - p-value: Significancia estadística

3. **Jensen-Shannon Divergence**: Mide diferencia entre distribuciones
   - 0 = distribuciones idénticas
   - 1 = distribuciones completamente diferentes

4. **Chi-cuadrado**: Para variables categóricas
   - Test de independencia
   - p-value: Significancia del cambio

In [ ]:
# Configurar umbrales
THRESHOLD_PSI = 0.2
THRESHOLD_KS = 0.2
THRESHOLD_JS = 0.2
THRESHOLD_CHI2 = 0.05

print('Umbrales configurados:')
print(f'  PSI: {THRESHOLD_PSI}')
print(f'  KS: {THRESHOLD_KS}')
print(f'  JS: {THRESHOLD_JS}')
print(f'  Chi2 (p-value): {THRESHOLD_CHI2}')

In [ ]:
# Calcular drift para todo el dataset
if 'baseline_data' in locals() and 'current_data' in locals():
    print('Calculando métricas de drift...')
    drift_results = detect_drift(
        baseline_data,
        current_data,
        threshold_psi=THRESHOLD_PSI,
        threshold_ks=THRESHOLD_KS,
        threshold_js=THRESHOLD_JS,
        threshold_chi2=THRESHOLD_CHI2
    )
    
    print('✅ Métricas de drift calculadas')
    print(f'\nTotal de variables analizadas: {len(drift_results)}')
else:
    print('❌ Error: Carga primero los datos baseline y actuales')

## 4. Resumen de Resultados

In [ ]:
# Resumen general
if 'drift_results' in locals():
    print('='*80)
    print('RESUMEN DE DRIFT DETECTION')
    print('='*80)
    
    total_vars = len(drift_results)
    no_drift = len(drift_results[drift_results['overall_status'] == 'no_drift'])
    moderate_drift = len(drift_results[drift_results['overall_status'] == 'moderate_drift'])
    significant_drift = len(drift_results[drift_results['overall_status'] == 'significant_drift'])
    
    print(f'\nTotal de variables: {total_vars}')
    print(f'  Sin drift: {no_drift} ({(no_drift/total_vars*100):.1f}%)')
    print(f'  Drift moderado: {moderate_drift} ({(moderate_drift/total_vars*100):.1f}%)')
    print(f'  Drift significativo: {significant_drift} ({(significant_drift/total_vars*100):.1f}%)')
    
    print('\n' + '='*80)
else:
    print('❌ Error: Ejecuta primero el cálculo de drift')

In [ ]:
# Mostrar tabla completa de resultados
if 'drift_results' in locals():
    display(drift_results)

## 5. Alertas y Variables Críticas

In [ ]:
# Variables con drift significativo
if 'drift_results' in locals():
    significant_vars = drift_results[drift_results['overall_status'] == 'significant_drift']
    
    if len(significant_vars) > 0:
        print('🚨 ALERTA: Variables con drift significativo:')
        print('='*80)
        display(significant_vars[['variable', 'type', 'psi', 'ks_statistic', 'js_divergence', 'chi2_pvalue']])
        print('\n⚠️ RECOMENDACIÓN: Considerar retraining del modelo')
    else:
        print('✅ No se detectaron variables con drift significativo')
    
    # Variables con drift moderado
    moderate_vars = drift_results[drift_results['overall_status'] == 'moderate_drift']
    
    if len(moderate_vars) > 0:
        print('\n⚠️ Variables con drift moderado:')
        print('='*80)
        display(moderate_vars[['variable', 'type', 'psi', 'ks_statistic', 'js_divergence', 'chi2_pvalue']])
        print('\n💡 RECOMENDACIÓN: Monitorear estas variables')
else:
    print('❌ Error: Ejecuta primero el cálculo de drift')

## 6. Visualizaciones

In [ ]:
# Gráfico de estado por variable
if 'drift_results' in locals():
    # Gráfico de barras con estado
    fig, ax = plt.subplots(figsize=(12, 6))
    
    status_counts = drift_results['overall_status'].value_counts()
    colors = {'no_drift': 'green', 'moderate_drift': 'orange', 'significant_drift': 'red'}
    
    bars = ax.bar(status_counts.index, status_counts.values, 
                  color=[colors.get(status, 'gray') for status in status_counts.index])
    
    ax.set_xlabel('Estado de Drift')
    ax.set_ylabel('Número de Variables')
    ax.set_title('Distribución de Estados de Drift')
    ax.grid(axis='y', alpha=0.3)
    
    # Agregar valores en las barras
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom')
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print('❌ Error: Ejecuta primero el cálculo de drift')

In [ ]:
# Gráfico de métricas PSI por variable
if 'drift_results' in locals():
    # Filtrar variables numéricas con PSI
    numeric_vars = drift_results[
        (drift_results['type'] == 'numeric') & 
        (drift_results['psi'].notna())
    ].sort_values('psi', ascending=False)
    
    if len(numeric_vars) > 0:
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Colores según estado
        colors = []
        for status in numeric_vars['psi_status']:
            if status == 'no_drift':
                colors.append('green')
            elif status == 'moderate_drift':
                colors.append('orange')
            elif status == 'significant_drift':
                colors.append('red')
            else:
                colors.append('gray')
        
        bars = ax.barh(numeric_vars['variable'], numeric_vars['psi'], color=colors)
        
        # Línea de umbral
        ax.axvline(x=THRESHOLD_PSI, color='red', linestyle='--', 
                   label=f'Umbral PSI = {THRESHOLD_PSI}')
        
        ax.set_xlabel('PSI')
        ax.set_ylabel('Variable')
        ax.set_title('Population Stability Index (PSI) por Variable')
        ax.legend()
        ax.grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        print('No hay variables numéricas con PSI calculado')
else:
    print('❌ Error: Ejecuta primero el cálculo de drift')

In [ ]:
# Gráficos de distribución para variables con drift
if 'drift_results' in locals() and 'baseline_data' in locals() and 'current_data' in locals():
    # Obtener variables con drift significativo o moderado
    vars_with_drift = drift_results[
        drift_results['overall_status'].isin(['moderate_drift', 'significant_drift'])
    ]['variable'].head(6)  # Solo las primeras 6
    
    if len(vars_with_drift) > 0:
        n_vars = len(vars_with_drift)
        n_cols = 3
        n_rows = (n_vars + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten() if n_vars > 1 else [axes]
        
        for idx, var in enumerate(vars_with_drift):
            if var in baseline_data.columns and var in current_data.columns:
                ax = axes[idx]
                
                baseline_var = baseline_data[var].dropna()
                current_var = current_data[var].dropna()
                
                if pd.api.types.is_numeric_dtype(baseline_data[var]):
                    # Histograma para variables numéricas
                    ax.hist(baseline_var, bins=30, alpha=0.7, label='Baseline', density=True)
                    ax.hist(current_var, bins=30, alpha=0.7, label='Actual', density=True)
                    ax.set_xlabel(var)
                    ax.set_ylabel('Densidad')
                    ax.set_title(f'Distribución de {var}')
                    ax.legend()
                    ax.grid(alpha=0.3)
                else:
                    # Gráfico de barras para categóricas
                    baseline_counts = pd.Series(baseline_var).value_counts().head(10)
                    current_counts = pd.Series(current_var).value_counts().head(10)
                    
                    x = np.arange(len(baseline_counts))
                    width = 0.35
                    
                    ax.bar(x - width/2, baseline_counts.values, width, label='Baseline', alpha=0.7)
                    ax.bar(x + width/2, current_counts.values, width, label='Actual', alpha=0.7)
                    
                    ax.set_xlabel(var)
                    ax.set_ylabel('Frecuencia')
                    ax.set_title(f'Distribución de {var}')
                    ax.set_xticks(x)
                    ax.set_xticklabels(baseline_counts.index, rotation=45, ha='right')
                    ax.legend()
                    ax.grid(axis='y', alpha=0.3)
        
        # Ocultar ejes vacíos
        for idx in range(len(vars_with_drift), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.show()
    else:
        print('No hay variables con drift para visualizar')
else:
    print('❌ Error: Ejecuta primero el cálculo de drift')

## 7. Recomendaciones

In [ ]:
# Generar recomendaciones
if 'drift_results' in locals():
    significant_vars = drift_results[drift_results['overall_status'] == 'significant_drift']
    moderate_vars = drift_results[drift_results['overall_status'] == 'moderate_drift']
    
    print('='*80)
    print('RECOMENDACIONES')
    print('='*80)
    
    if len(significant_vars) > 0:
        print('\n🚨 ACCIONES INMEDIATAS:')
        print('  1. Revisar las variables con drift significativo')
        print('  2. Investigar causas del cambio en la distribución')
        print('  3. Considerar retraining del modelo')
        print('  4. Actualizar el dataset baseline si el cambio es válido')
        print(f'\n  Variables a revisar: {significant_vars["variable"].tolist()}')
    
    if len(moderate_vars) > 0:
        print('\n⚠️ ACCIONES RECOMENDADAS:')
        print('  1. Monitorear las variables con drift moderado')
        print('  2. Revisar tendencias a lo largo del tiempo')
        print('  3. Considerar ajustes menores en el modelo')
        print(f'\n  Variables a monitorear: {moderate_vars["variable"].tolist()}')
    
    if len(significant_vars) == 0 and len(moderate_vars) == 0:
        print('\n✅ ESTADO ACTUAL:')
        print('  - No se detectaron problemas significativos de drift')
        print('  - El modelo está funcionando con datos consistentes')
        print('  - Continuar con monitoreo regular')
    
    print('\n' + '='*80)
else:
    print('❌ Error: Ejecuta primero el cálculo de drift')

## 8. Guardar Resultados

In [ ]:
# Guardar resultados de drift
if 'drift_results' in locals():
    drift_results.to_csv('../../drift_results.csv', index=False)
    print('✅ Resultados de drift guardados en drift_results.csv')
    
    # Guardar resumen
    summary = {
        'total_variables': len(drift_results),
        'no_drift': len(drift_results[drift_results['overall_status'] == 'no_drift']),
        'moderate_drift': len(drift_results[drift_results['overall_status'] == 'moderate_drift']),
        'significant_drift': len(drift_results[drift_results['overall_status'] == 'significant_drift'])
    }
    
    import json
    with open('../../drift_summary.json', 'w') as f:
        json.dump(summary, f, indent=2)
    
    print('✅ Resumen guardado en drift_summary.json')
else:
    print('❌ Error: Ejecuta primero el cálculo de drift')

## 9. Aplicación Streamlit

Para ejecutar la aplicación Streamlit interactiva:

```bash
streamlit run streamlit_monitoring_app.py
```

La aplicación Streamlit permite:

1. **Cargar datos**: Baseline y actuales
2. **Configurar umbrales**: Ajustar parámetros de drift
3. **Visualizar métricas**: Tablas y gráficos interactivos
4. **Ver alertas**: Variables con drift significativo
5. **Analizar distribuciones**: Comparación baseline vs actual
6. **Recomendaciones**: Sugerencias automáticas

---